# MOTIS API Notebook: Find Connections from A to B

This Jupyter notebook demonstrates how to use the **MOTIS routing API** to compute public transit, walking, biking, and multimodal connections between two locations. We will use the `/api/v3/plan` endpoint for optimal routing, and also show examples of geocoding, timetables, and map integration.

## API Endpoints Covered
- `routing/plan`: Finds optimal connections from A to B
- `geocode/geocode`: Autocomplete and resolve place names
- `geocode/reverseGeocode`: Get place info from coordinates
- `timetable/stoptimes`: Get departures from a stop
- `map/trips`: Get trips in a geographic area

We'll use Python with `requests` to interact with the API.

### Motivation
MOTIS is a powerful open-source multimodal routing engine built on OpenStreetMap and GTFS data. This notebook serves as a practical starting point for developers and analysts exploring transport networks.

In [1]:
# Import required libraries
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import display, HTML, JSON

## 1. Configure API Settings

Choose which server to use. You can run a local instance or use the public staging/production server.

In [2]:
# 🌐 API Base URLs
BASE_URLS = {
    "local": "http://localhost:8080",
    "production": "https://api.transitous.org",
    "staging": "https://staging.api.transitous.org"
}

# 🔧 Select your environment
ENV = "production"  # Change to 'local' or 'staging' if needed
BASE_URL = BASE_URLS[ENV]

# 👉 Main routing endpoint
PLAN_ENDPOINT = f"{BASE_URL}/api/v3/plan"
GEOCODE_ENDPOINT = f"{BASE_URL}/api/v1/geocode"
REVERSE_GEOCODE_ENDPOINT = f"{BASE_URL}/api/v1/reverse-geocode"
STOPTIMES_ENDPOINT = f"{BASE_URL}/api/v1/stoptimes"
MAP_TRIPS_ENDPOINT = f"{BASE_URL}/api/v1/map/trips"

print(f"Using {ENV.upper()} server: {BASE_URL}")

Using PRODUCTION server: https://api.transitous.org


## 2. Helper Functions

Reusable functions to interact with the API cleanly.

In [3]:
def geocode_place(text, language="en", type="STOP,ADDRESS", place=None):
    """
    Resolve a textual location (e.g. street name, stop) to coordinates.
    """
    params = {
        "text": text,
        "language": language,
        "type": type
    }
    if place:
        params["place"] = place

    response = requests.get(GEOCODE_ENDPOINT, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Geocode failed: {response.status_code} - {response.text}")
        return None

def reverse_geocode(lat, lon, type="STOP,ADDRESS"):
    """
    Get place information from coordinates.
    """
    params = {
        "place": f"{lat},{lon}",
        "type": type
    }
    response = requests.get(REVERSE_GEOCODE_ENDPOINT, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Reverse geocode failed: {response.status_code} - {response.text}")
        return None

def get_connections(from_place, to_place, 
                   time=None, arrive_by=False, max_transfers=4, 
                   transit_modes="TRANSIT", direct_modes="WALK",
                   num_itineraries=3, detailed_transfers=True):
    """
    Compute optimal connections from A to B using the plan endpoint.
    
    Parameters:
        from_place: str (lat,lon or stop ID)
        to_place: str (lat,lon or stop ID)
        time: ISO datetime string (default: now)
        arrive_by: bool (if True, plan to arrive at `time`)
        max_transfers: int
        transit_modes: comma-separated string (e.g., 'TRANSIT', 'METRO,BUS')
        direct_modes: mode for first/last mile (e.g., 'WALK', 'BIKE')
        num_itineraries: number of results to return
        detailed_transfers: include step-by-step instructions
    """
    params = {
        "fromPlace": from_place,
        "toPlace": to_place,
        "maxTransfers": max_transfers,
        "transitModes": transit_modes,
        "directModes": direct_modes,
        "numItineraries": num_itineraries,
        "detailedTransfers": detailed_transfers,
        "arriveBy": arrive_by,
        "timetableView": True
    }
    
    if time:
        params["time"] = time
    else:
        params["time"] = (datetime.now()).isoformat()

    response = requests.get(PLAN_ENDPOINT, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Routing failed: {response.status_code} - {response.text}")
        return None

def get_stoptimes(stop_id, n=5, arrive_by=False, direction="LATER"):
    """
    Get next departures or arrivals from a stop.
    """
    params = {
        "stopId": stop_id,
        "n": n,
        "arriveBy": arrive_by,
        "direction": direction
    }

    response = requests.get(STOPTIMES_ENDPOINT, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Stoptimes failed: {response.status_code} - {response.text}")
        return None

def get_trips_in_area(min_lat_lon, max_lat_lon, zoom=13, 
                      start_time=None, end_time=None):
    """
    Get all trips operating in a bounding box and time window.
    """
    if not start_time:
        start_time = (datetime.now()).isoformat()
    if not end_time:
        end_time = (datetime.now() + timedelta(hours=2)).isoformat()

    params = {
        "zoom": zoom,
        "min": min_lat_lon,
        "max": max_lat_lon,
        "startTime": start_time,
        "endTime": end_time
    }

    response = requests.get(MAP_TRIPS_ENDPOINT, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Map trips failed: {response.status_code} - {response.text}")
        return None

## 3. Example: Find a Connection from A to B

Let’s plan a trip from **Hauptbahnhof München** to **Olympiazentrum München** using transit + walking.

In [4]:
# 📍 Step 1: Geocode the start and end places

from_query = "München Hauptbahnhof"
to_query = "Olympiazentrum, München"

print(f"🔍 Geocoding '{from_query}'...")
from_results = geocode_place(from_query, type="STOP")
if not from_results:
    raise Exception("Could not geocode start location")

from_stop = from_results[0]
from_place = f"{from_stop['lat']},{from_stop['lon']}"

print(f"📍 Found: {from_stop['name']} (ID: {from_stop['id']}, {from_stop['lat']},{from_stop['lon']})")

print(f"\n🔍 Geocoding '{to_query}'...")
to_results = geocode_place(to_query, type="STOP")
if not to_results:
    raise Exception("Could not geocode end location")

to_stop = to_results[0]
to_place = f"{to_stop['lat']},{to_stop['lon']}"

print(f"📍 Found: {to_stop['name']} (ID: {to_stop['id']}, {to_stop['lat']},{to_stop['lon']})")

🔍 Geocoding 'München Hauptbahnhof'...
📍 Found: München Hauptbahnhof (ID: at-Railway-Current-Reference-Data-2025_de:09162:100:11:11, 48.13988,11.557491)

🔍 Geocoding 'Olympiazentrum, München'...
📍 Found: Olympiazentrum (ID: de-DELFI_de:09162:350, 48.178894,11.55617)


In [15]:
# ⏱️ Set travel time
travel_time = (datetime.now() + timedelta(minutes=10)).isoformat()

print(f"🚆 Planning journey from {from_stop['name']} to {to_stop['name']} at {travel_time}...\n")

# 🚆 Get connections
itineraries = get_connections(
    from_place=from_place,
    to_place=to_place,
    time=travel_time,
    arrive_by=False,
    max_transfers=3,
    transit_modes="TRANSIT",
    direct_modes="WALK",
    num_itineraries=5
)

if not itineraries:
    raise Exception("No connections found")

print(itineraries)

🚆 Planning journey from München Hauptbahnhof to Olympiazentrum at 2025-08-12T05:02:11.086556...

{'requestParameters': {}, 'debugOutput': {'direct': 3, 'execute_time': 182, 'fastest_direct': 32767, 'fp_update_prevented_by_lower_bound': 134451, 'interval_extensions': 0, 'lb_time': 187, 'n_dest_offsets': 22, 'n_earliest_arrival_updated_by_footpath': 223322, 'n_earliest_arrival_updated_by_route': 346019, 'n_earliest_trip_calls': 7456342, 'n_footpaths_visited': 1888891, 'n_routes_visited': 734363, 'n_routing_time': 0, 'n_start_offsets': 130, 'n_td_dest_offsets': 0, 'n_td_start_offsets': 0, 'query_preparation': 77, 'route_update_prevented_by_lower_bound': 0}, 'from': {'name': 'START', 'lat': 48.13988, 'lon': 11.557491, 'level': 0.0, 'vertexType': 'NORMAL'}, 'to': {'name': 'END', 'lat': 48.178894, 'lon': 11.55617, 'level': 0.0, 'vertexType': 'NORMAL'}, 'direct': [], 'itineraries': [], 'previousPageCursor': 'EARLIER|1754870400', 'nextPageCursor': 'LATER|1754870400'}


## 4. Display Itineraries

Parse and visualize the returned routes.

In [6]:
def format_duration(seconds):
    """Convert seconds to HH:MM format."""
    return str(timedelta(seconds=seconds)).split('.')[0]

def parse_itinerary(iti, idx):
    start = datetime.fromisoformat(iti['startTime'].replace("Z", "+00:00")).strftime("%H:%M")
    end = datetime.fromisoformat(iti['endTime'].replace("Z", "+00:00")).strftime("%H:%M")
    duration = format_duration(iti['duration'])
    transfers = iti['transfers']
    
    legs = []
    for leg_idx, leg in enumerate(iti['legs']):
        mode = leg['mode']
        from_name = leg['from']['name']
        to_name = leg['to']['name']
        headsign = leg.get('headsign', '')
        route = leg.get('routeShortName', '')
        
        if mode == "WALK":
            legs.append(f"🚶 Walk from <strong>{from_name}</strong> to <strong>{to_name}</strong>")
        elif mode in ["BUS", "TRAM", "SUBWAY", "RAIL"]:
            legs.append(f"🚍 {mode} <strong>{route}</strong> to <strong>{to_name}</strong> (headsign: {headsign})")
        else:
            legs.append(f"{mode} from {from_name} to {to_name}")
    
    return f"<h4>Option {idx+1}: {start} → {end} | {duration} | {transfers} transfer(s)</h4><ul><li>{'</li><li>'.join(legs)}</ul>"

In [14]:
# 📋 Display top itineraries
print("✅ Found routes:\n")

print(itineraries['itineraries'])

✅ Found routes:

[]


## 5. View Raw JSON Response (Optional)

Uncomment to explore full API response structure.

In [8]:
# 🔍 View full API response
# JSON(itineraries)

## 6. Check Departures from Origin Stop

Get real-time stoptimes for the departure location.

In [9]:
# 🕒 Get next departures from München Hbf
stop_times = get_stoptimes(stop_id=from_stop['id'], n=5, arrive_by=False)

if stop_times and 'stopTimes' in stop_times:
    df = pd.DataFrame([
        {
            'Time': datetime.fromisoformat(st['place']['departure'].replace("Z", "+00:00")).strftime("%H:%M"),
            'Mode': st['mode'],
            'Route': st.get('routeShortName', 'N/A'),
            'Destination': st['headsign'],
            'Cancelled': "Yes" if st.get('cancelled', False) else "No"
        }
        for st in stop_times['stopTimes']
    ])
    print(f"🚌 Next departures from {from_stop['name']}:\n")
    display(df)
else:
    print("No departure data available.")

🚌 Next departures from München Hauptbahnhof:



,Time,Mode,Route,Destination,Cancelled
0,04:54,REGIONAL_FAST_RAIL,D99,Zürich HB,No
1,05:22,REGIONAL_FAST_RAIL,RJX 61,Budapest-Keleti,No
2,05:28,REGIONAL_FAST_RAIL,NJ 40491,Innsbruck Hauptbahnhof,No
3,05:28,REGIONAL_FAST_RAIL,NJ 421,Innsbruck Hauptbahnhof,No
4,05:49,REGIONAL_FAST_RAIL,WB 961,Wien Westbahnhof,No


## 7. View Routes on Map Area

Fetch all trips operating in a general area around the destination.

In [10]:
# 🗺️ Get trips in the area around Olympiazentrum
buffer = 0.01  # ~1km around
min_lat_lon = f"{to_stop['lat'] - buffer},{to_stop['lon'] - buffer}"
max_lat_lon = f"{to_stop['lat'] + buffer},{to_stop['lon'] + buffer}"

trips_in_area = get_trips_in_area(
    min_lat_lon=min_lat_lon,
    max_lat_lon=max_lat_lon,
    zoom=15
)

if trips_in_area:
    print(f"📍 Trips operating near {to_stop['name']}:\n")
    for trip_group in trips_in_area:
        mode = trip_group['mode']
        route_color = trip_group.get('routeColor', 'N/A')
        trips = [t['routeShortName'] for t in trip_group['trips']]
        print(f"- {mode} (color: {route_color}): {', '.join(trips)}")
else:
    print("No trips found in area.")

No trips found in area.


## 8. Reverse Geocode Example

Get place information from coordinates directly.

In [11]:
# 📍 Reverse geocode Olympiazentrum's coordinates
rev = reverse_geocode(to_stop['lat'], to_stop['lon'], type="STOP,ADDRESS")

if rev:
    print(f"Reverse geocode result:\n")
    for match in rev[:3]:  # Show top 3 results
        print(f"- {match['name']} ({match['type']}) @ {match['lat']:.4f},{match['lon']:.4f}")
else:
    print("No reverse geocode result.")

❌ Reverse geocode failed: 500 - {"error":"enum LocationTypeEnum: unknown value STOP,ADDRESS"}
No reverse geocode result.


## Summary

This notebook demonstrated how to:

✅ Geocode place names to coordinates

✅ Find multimodal connections between two points using `/api/v3/plan`

✅ Display itineraries in a readable format

✅ Fetch real-time stoptimes for a station

✅ Explore all trips in a geographic bounding box

✅ Reverse geocode coordinates

You can extend this to support bike routing, wheelchair accessibility, fare calculation, and more by adjusting the request parameters.

### Useful Parameters to Explore Next
- `requireBikeTransport=true`
- `pedestrianProfile=WHEELCHAIR`
- `transitModes=SUBWAY,BUS`
- `withFares=true`
- `via=stopId1,stopId2`

### Resources
- **MOTIS GitHub**: [https://github.com/motis-project/motis](https://github.com/motis-project/motis)
- **API Docs**: [https://redocly.com/redoc/](https://redocly.com/redoc/)
- **OpenAPI Spec**: [https://raw.githubusercontent.com/motis-project/motis/master/openapi.yaml](https://raw.githubusercontent.com/motis-project/motis/master/openapi.yaml)

**Note**: If you're running locally, ensure MOTIS is running on `localhost:8080` with GTFS data loaded.

Happy routing! 🚇🚌🚶‍♂️